# Fan-beam tomography: flat detector

This page describes fan-beam tomographic image reconstruction
using the Julia package
[`Sinograms.jl`](https://github.com/JuliaImageRecon/Sinograms.jl).

This page focuses on fan-beam with a "flat" detector,
i.e., one row of a flat-panel detector
as used in many cone-beam CT (CBCT) systems.

### Setup

Packages needed here.

In [ ]:
using Plots: plot, gui
using Unitful: cm
using Sinograms: SinoFanFlat, rays, plan_fbp, Window, Hamming, fbp, sino_geom_plot!
using ImageGeoms: ImageGeom, fovs, MaskCircle
using ImagePhantoms: SheppLogan, shepp_logan, radon, phantom
using MIRTjim: jim, prompt

The following line is helpful when running this file as a script;
this way it will prompt user to hit a key after each figure is displayed.

In [ ]:
isinteractive() ? jim(:prompt, true) : prompt(:draw);

## Fan-beam sinogram of Shepp-Logan phantom

For illustration,
we start by synthesizing
a fan-beam sinogram
of the Shepp-Logan phantom.

For completeness,
we use units (from Unitful),
but units are optional.

Use `ImageGeom` to define the image geometry.

In [ ]:
ig = ImageGeom(MaskCircle(); dims=(128,126), deltas = (0.2cm,0.2cm) )

Use `SinoFanFlat` to define the sinogram geometry.

In [ ]:
rg = SinoFanFlat( ; nb = 130, d = 0.3cm, na = 100, dsd = 50cm, dod = 14cm)

Examine the geometry to verify the FOV:

In [ ]:
jim(axes(ig), ig.mask; prompt=false)
sino_geom_plot!(rg, ig)

In [ ]:
prompt()

Ellipse parameters for Shepp-Logan phantom:

In [ ]:
μ = 0.1 / cm # typical linear attenuation coefficient
ob = shepp_logan(SheppLogan(); fovs = fovs(ig), u = (1, 1, μ))

Arc fan-beam sinogram for Shepp-Logan phantom:

In [ ]:
sino = radon(rays(rg), ob)
jim(axes(rg), sino; title="Shepp-Logan sinogram", xlabel="r", ylabel="ϕ")

## Image reconstruction via FBP
Here we start with a "plan",
which would save work if we were reconstructing many images.
For illustration we include `Hamming` window.

In [ ]:
plan = plan_fbp(rg, ig; window = Window(Hamming(), 1.0))
fbp_image = fbp(plan, sino)

A narrow color window is needed to see the soft tissue structures:

In [ ]:
clim = (0.9, 1.1) .* μ
jim(axes(ig), fbp_image, "FBP image for flat case"; clim)

For comparison, here is the ideal phantom image

In [ ]:
true_image = phantom(axes(ig)..., ob, 2)
jim(axes(ig)..., true_image, "True phantom image"; clim)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*